<a href="https://colab.research.google.com/github/ArstmN/Pytorch_DL_FaraDars/blob/main/Saving_Loading_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preparation**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64)



# **Model Implementation**

In [ ]:
class NNModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(in_features=784, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.sequential(x)
        return x

model = NNModel()

# **Setting Loss Function**

In [ ]:
# loss function = criterion
loss_fn = nn.CrossEntropyLoss() 

# **Setting Hyperparameters**

In [ ]:
learning_rate = 1e-3
batch_size = 64
n_epochs = 3

# **Setting the Optimizer**

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# **Optimization Loop**

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(dataloader):

        # generate predictions
        pred = model(X)

        # calculate loss
        loss = loss_fn(pred, y)

        # reset gradients
        optimizer.zero_grad()

        # compute gradients
        loss.backward()

        # update parameters w, b
        optimizer.step()

        if batch % 100 == 0:
            loss = loss.item()
            current = batch * len(X)
            print(f"Loss: {loss:>.4f} --- [{current:>5d} out of {len(dataloader.dataset):>5d}]")


def test(dataloader, model, loss_fn):
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for X, y in dataloader:

            pred = model(X) 

            loss = loss_fn(pred, y)

            test_loss += loss.item()
            correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()

        test_loss /= len(dataloader.dataset)
        correct /= len(dataloader.dataset)

        print(f"\nResults: Accuracy: {(100 * correct):>0.1f}%, Average Loss: {test_loss:>8f} \n")

In [ ]:
for epoch in range(n_epochs):
    print(f"[Epoch {epoch+1}]\n")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Completed!')

[Epoch 1]

Loss: 1.8311 --- [    0 out of 60000]
Loss: 1.7797 --- [ 6400 out of 60000]
Loss: 1.8021 --- [12800 out of 60000]
Loss: 1.6734 --- [19200 out of 60000]
Loss: 1.6928 --- [25600 out of 60000]
Loss: 1.7037 --- [32000 out of 60000]
Loss: 1.5829 --- [38400 out of 60000]
Loss: 1.5051 --- [44800 out of 60000]
Loss: 1.5416 --- [51200 out of 60000]
Loss: 1.5011 --- [57600 out of 60000]

Results: Accuracy: 56.9%, Average Loss: 0.023817 

[Epoch 2]

Loss: 1.4991 --- [    0 out of 60000]
Loss: 1.4958 --- [ 6400 out of 60000]
Loss: 1.4890 --- [12800 out of 60000]
Loss: 1.3103 --- [19200 out of 60000]
Loss: 1.4227 --- [25600 out of 60000]
Loss: 1.3395 --- [32000 out of 60000]
Loss: 1.3291 --- [38400 out of 60000]
Loss: 1.2618 --- [44800 out of 60000]
Loss: 1.2610 --- [51200 out of 60000]
Loss: 1.3463 --- [57600 out of 60000]

Results: Accuracy: 59.8%, Average Loss: 0.020200 

[Epoch 3]

Loss: 1.2217 --- [    0 out of 60000]
Loss: 1.3074 --- [ 6400 out of 60000]
Loss: 1.3457 --- [12800 out

# **Model Saving and Loading**


## Saving and Loading Model Weights

In [ ]:
torch.save(model.state_dict(), "model_weights.pth")

In [ ]:
model_1 = NNModel()
model_1.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

## Saving and Loading Models with Shapes

In [ ]:
torch.save(model, 'model_shapes.pth')

model_2 = torch.load('model_shapes.pth')

# **Making Predictions**


In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [ ]:
model_1.eval()

x, y = test_data[2][0], test_data[2][1]

with torch.no_grad():
    pred = model_1(x)
    predicted, actual = classes[pred[0].argmax(dim=0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Trouser"
